### Import packages

In [ ]:
import math
import os
import random
import time
import pandas as pd
import geopandas as gpd
from owslib.wms import WebMapService
from pyproj import Proj, transform
import tqdm
import multiprocess as mp

### Read csv file
- read csv file that contains grids (top, left, bottom, right extent data)

In [ ]:
csv_dir = "data/gov_data_for_downloading_sp_06.csv"

CRS = 'EPSG:4326' # CRS for gov csv file

#image parmeters - PX_WIDTH: number of pixels horizontally
PX_WIDTH = 512
PX_HEIGHT = 512

download_folder = "data/downloaded_images"

In [ ]:
# read csv file
df = pd.read_csv(csv_dir)
df

# Parameters

In [ ]:
#get secret from key vault
#credential information are updated on 3/1/2022
sat_id = 'unicef_innovation'
sat_pass = 'R4HyxsnxMu$sDju'
sat_connect_id = '5f4b5f3d-47e0-402a-996a-ed37f05a871b'

#prepare wms connection strings
wms = WebMapService('http://evwhs.digitalglobe.com/mapservice/wmsaccess?connectid=' + sat_connect_id, 
                    version='1.1.1', 
                    username=sat_id,
                    password=sat_pass)

# prepare ID for each image
tile_id_col = 'point_id'
ID = df[tile_id_col]
size_data = len(ID)

# folder for image download
DIRECTORY = download_folder # os.path.join(download_folder, "")

#don't change anything below this
FORMAT      = 'image/png'
LAYERS      = ['DigitalGlobe:Imagery']
SIZE        = (PX_WIDTH, PX_HEIGHT)
SRS         = CRS
TRANSPARENT = True
REQUEST='GetMap'
STYLES=''
EXCEPTIONS  = 'application/vnd.ogc.se_xml'
FEATUREPROFILE='Most_Aesthetic_Mosaic_Profile'
COVERAGE_CQL_FILTER='productType =%27Pan Sharpened Natural Color%27'
COVERAGE_CQL_FILTER=''

## Function for image download for multiproecssing

In [ ]:
def img_down(no):
    
    ID = str(df[tile_id_col][no])
    
    if os.path.exists(os.path.join(download_folder, str(ID) +'.png')):
        return
    
    try:
        bbox=(df['left'][no],df['bottom'][no],df['right'][no],df['top'][no])
        
        img = wms.getmap(
                    layers=LAYERS,
                    srs=SRS,
                    bbox=bbox,
                    size=SIZE,
                    featureProfile=FEATUREPROFILE,
                    coverage_cql_filter=COVERAGE_CQL_FILTER,
                    exceptions=EXCEPTIONS,
                    transparent=TRANSPARENT,
                    format=FORMAT          
        )
        
        if(img.info()['Content-Disposition'].endswith('.png')):
            with open(os.path.join(download_folder, str(ID) +'.png'), 'wb') as png:
                png.write(img.read())
    
    except Exception as e:
            print(e, "Error downloading:", ID)

## Download

In [ ]:
# create output folder
os.makedirs(download_folder, exist_ok = True)

#generate numbers between 0 and data size
tile_nums=range(size_data)

#call the downloading function with record numbers as inputs
#change the number of processes to smaller number if memory problem occurs
p = mp.Pool()
results = list(tqdm.tqdm(p.imap_unordered(img_down, tile_nums), total=size_data))

p.close()
p.join()